In [1]:
import numpy as np
from lume_epics.epics_server import Server
from lume_epics.model import SurrogateModel

from lume_model.variables import ScalarInputVariable, ScalarOutputVariable, ImageInputVariable, ImageOutputVariable

In [2]:
class DemoModel(SurrogateModel):
    input_variables = {
        "input1": ScalarInputVariable(name="input1", value=1, range=[0, 244]),
        "input2": ScalarInputVariable(name="input2", value=2, range=[0, 244]),
    }

    output_variables = {
        "output1": ImageOutputVariable(
            name="output1", axis_labels=["count_1", "count_2"], x_min=0, x_max=5, y_min=0, y_max=5
        )
    }
    
    def predict(self, input_variables):
        self.output_variables["output1"].value = np.random.uniform(self.input_variables["input1"].value, # lower dist bound
                                                                   self.input_variables["input2"].value, # upper dist bound
                                                                   (50,50)
                                                                  ) #shape
        

        return list(self.output_variables.values())

In [3]:
prefix = "test"
server = Server(DemoModel, prefix)
server.start_pva_server()
server.start_ca_server()

Running model...Ellapsed time: 0.0002617835998535156


In [4]:
from lume_epics.client.widgets.plots import ImagePlot
from lume_epics.client.widgets.sliders import build_sliders
from lume_epics.client.controller import Controller

from bokeh.io import output_notebook, show
from bokeh import palettes
from bokeh.layouts import column, row

# load bokeh
output_notebook()

Loading BokehJS ...

In [ ]:
server.stop()

In [5]:
 # build sliders for the command process variable database
inputs = list(DemoModel.input_variables.values())
controller = Controller("pva")
sliders = build_sliders(inputs, controller, prefix)

In [6]:
# Create custom palette with low values set to white
pal = list(palettes.viridis(244))  # 256 - 12 (set lowest 5% to white)
pal = ["#FFFFFF"] * 12 + pal
pal = tuple(pal)

# Set up image update callback
def image_update_callback():
    """
    Calls plot controller update with the current global process variable
    """
    global current_image_pv
    image_plot.update(current_image_pv)

# create plot
output_variables = list(DemoModel.output_variables.values())
image_plot = ImagePlot(output_variables, controller, prefix)
image_plot.build_plot(pal)

KeyError: 'dw'

In [ ]:
def render_app(doc):
    """
    Function for rendering the application within the embedded bokeh server.
    """
    doc.title = "Demo App"
    doc.add_root( column(
        row(column(sliders, width=350), column(image_plot.plot, height=300)),  # add sliders
    ))
    doc.add_periodic_callback(image_update_callback, 250)
    doc.add_periodic_callback(striptool_update_callback, 250)
    doc.add_periodic_callback(table_update_callback, 250)

    
show(render_app)